# Домашнее задание. Ансамблирование моделей

- Теперь решаем задачу регрессии - предскажем цены на недвижимость. Использовать датасет https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
- Данных немного, поэтому необходимо использовать 10-fold кросс-валидацию для оценки качества моделей
- Построить случайный лес, вывести важность признаков
- Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
- Для валидации модели 2-го уровня использовать отдельный hold-out датасет, как на занятии
- Показать, что использование ансамблей моделей действительно улучшает качество (стекинг vs другие модели сравнивать на hold-out)
- В качестве решения: Jupyter notebook с кодом, комментариями и графиками, ссылка на гитхаб

## 0. Библиотеки

In [71]:
# Для работы с данными
import pandas as pd
import numpy as np

#Для работы с моделями
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

#Для визуализации
import matplotlib.pyplot as plt
%matplotlib inline

from jupyterthemes import jtplot
jtplot.style()

## 1. Загрузка данных

In [39]:
data = pd.read_csv('data/train.csv')

In [40]:
# SalePrice - target value
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### 1.1 Замена пустых значений и работа с категориальными признаками

In [41]:
# Находим не категориальные признаки
cat_feat = list(data.dtypes[data.dtypes == object].index)
data[cat_feat] = data[cat_feat].fillna('nan')

cat_nunique = data[cat_feat].nunique()

cat_feat = list(cat_nunique[cat_nunique < 30].index)

# Не категориальные признаки
num_feat = [f for f in data if f not in (cat_feat + ['ID', 'target'])]

In [42]:
# Создаем дамми-переменные для категорий
dummy_data = pd.get_dummies(data[cat_feat], columns=cat_feat)

dummy_cols = list(set(dummy_data))

dummy_data = dummy_data[dummy_cols]

train = pd.concat([data[num_feat].fillna(-999),
                     dummy_data], axis=1)

### 1.2 Обучающие и тестовые выборки

In [43]:
X = train.loc[:, train.columns != 'SalePrice']
y = train['SalePrice']

## 2. Модель - Случайный лес. Анализ важности признаков

In [44]:
reg_rf = RandomForestRegressor(n_estimators=10, max_depth=5, min_samples_leaf=20, max_features=0.5, n_jobs=-1)

In [45]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(reg_rf, X, y, cv=10)
#оценка модели с помощью кросс-валидации на 10-fold
score.mean()

0.813778553994197

In [46]:
#Зафитить модель
reg_rf.fit(X,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features=0.5, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=20, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [47]:
#Важность признаков для модели
Columns = pd.DataFrame(X.columns, columns=['column'])
Columns['importance'] = pd.Series(reg_rf.feature_importances_)
#Выбираем только значимые признаки для модели и сортируем их в порядке значимости
Columns[(Columns['importance']>0)].sort_values('importance', ascending=False).style.bar()

,column,importance
4,OverallQual,0.315917
214,ExterQual_TA,0.184464
16,GrLivArea,0.118233
6,YearBuilt,0.0856758
12,TotalBsmtSF,0.0660777
106,BsmtQual_Ex,0.0437851
3,LotArea,0.0354575
13,1stFlrSF,0.035187
41,ExterQual_Gd,0.0238649
9,BsmtFinSF1,0.012952


## 3. Стекинг из трех моделей -  1 линейную модель и 1 нелинейную

Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
Для валидации модели 2-го уровня использовать отдельный hold-out датасет, как на занятии

In [72]:
#Модели для получения мета-признаков
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

#Инструмент для разбиения на фолды
from sklearn.model_selection import StratifiedKFold

In [73]:
# Три модели для стекинга
stack_models = [  # Случайный лес
                 RandomForestRegressor(),
                  # Логистическая регрессия
                 LogisticRegression(),
                  # Линейная регрессия
                 LinearRegression()
                ]

In [74]:
temp = RandomForestRegressor()

In [75]:
# Создание обучающих и тестовых выборок
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [95]:
# Функция для получения мета-признаков
def get_meta_features(clf, X_train, y_train, X_test, stack_cv):
    meta_train = np.zeros_like(y_train, dtype=float)
    meta_test = np.zeros_like(y_test, dtype=float)
    
    #print(X_train, y_train)
    for i, (train_ind, test_ind) in enumerate(stack_cv.split(X_train, y_train)):
        clf.fit(X_train.iloc[train_ind], y_train.iloc[train_ind])
        meta_train[test_ind] = clf.predict(X_train.iloc[test_ind])
        meta_test += clf.predict(X_test)
    
    return meta_train, meta_test / stack_cv.n_splits

In [96]:
# Разбиваем на фолды
stack_cv = StratifiedKFold(n_splits=4, random_state=555)

In [97]:
# Получаем метапризнаки для мета-модели
meta_train = []
meta_test = []
col_names = []

# Создание мета-признаков
for i in stack_models:
    print("Обучение и создание признаков для модели {0}".format(i.__class__.__name__))
    meta_train_new, meta_test_new =  get_meta_features(i, X_train, y_train, X_test, stack_cv)
    meta_train.append(meta_train_new)
    meta_test.append(meta_test_new)
    col_names.append(i.__class__.__name__)
    

Обучение и создание признаков для модели RandomForestRegressor


/Users/akirichenko/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


Обучение и создание признаков для модели LogisticRegression


/Users/akirichenko/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


Обучение и создание признаков для модели LinearRegression


In [108]:
X_meta_train = pd.DataFrame({col_names[i]: meta_train[i] for i in range(len(col_names))})
X_meta_test = pd.DataFrame({col_names[i]: meta_test[i] for i in range(len(col_names))})

In [115]:
# Обучение мета-моделей
for i in stack_models:
    i.fit(X_meta_train, y_train)
    score = i.score(X_meta_test, y_test)
    print('Оценка мета-модели {0} : {1}'.format(i.__class__.__name__, score))


Оценка мета-модели RandomForestRegressor : 0.7586161233168023
Оценка мета-модели LogisticRegression : 0.00684931506849315
Оценка мета-модели LinearRegression : 0.8957097581013334
